In [1]:
# Import the libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Import data

if __name__ == "__main__":
    import pyreadr
    import os

    pd.set_option("display.precision", 3)
    data_dir_path = os.getcwd() + '/dataverse_files/'

    croissance_et_climat_decadaires = pyreadr.read_r(data_dir_path + "croissance_et_climat_decadaires.rds" ).popitem()[1]
    valorisation_annuelle = pyreadr.read_r(data_dir_path + "valorisation_annuelle.rds" ).popitem()[1]

## Data Preparation

### Growths and decadal climates

In [14]:
identifier_columns = ["ucs", "safran", "sol", "type_de_prairie", "gestion"]
valuable_columns = identifier_columns + ["annee", "decade", "Tmin", "Tmax", "Tmoy", "Rain", "RG", "im", "croissance"]
df = croissance_et_climat_decadaires.loc[:, valuable_columns]

annee_inf = df["annee"].max() - 5
df = df.loc[(df["annee"] > annee_inf), :]

df.sort_values(by= identifier_columns + ["annee", "decade"], inplace=True)
df.drop("type_de_prairie", axis= 'columns', inplace= True)
identifier_columns.remove("type_de_prairie")

df.dropna(inplace= True)
data_preserved = (df.isna() | df.isnull()).sum().sum() == 0
data_preserved = data_preserved & (df.groupby(identifier_columns).count()["decade"] % 37 == 0).all()
assert data_preserved == True

df["annee"] = df["annee"].astype("int64")
df["decade"] = df["decade"].astype("int64")

croissance_et_climat_decadaires_preprocessed = df

### Annual valuations

In [15]:
valuable_columns = ["ucs", "safran", "sol", "gestion", "annee", "cumul_croissance"]
df = valorisation_annuelle.loc[:, valuable_columns]

df = df.reset_index(drop=True)
df[["annee"]] = df[["annee"]].astype("int64")

annee_inf = df["annee"].max() - 5
df = df.loc[(df["annee"] > annee_inf), :]

valorisation_annuelle_preprocessed = df

### Concatenation

In [17]:
data_set = croissance_et_climat_decadaires_preprocessed.merge(valorisation_annuelle_preprocessed, how='left', on=["ucs", "safran","sol","gestion","annee"])

assert (data_set.isna() | data_set.isnull()).sum().sum() == 0

data_set_Y = pd.DataFrame({'croissance' : data_set.pop('croissance')})
data_set_X = data_set

## Data Preprocessing

In [34]:
X, Y = list(), list()

last_progression = 0
window_size = 3
batch_size = 37*1 # number of decades per year times number of years, per ID
for batch_index in range(0, len(data_set) - batch_size, batch_size): # for each batch
    batch_end_index = batch_index + batch_size
    for index in range(batch_index, batch_end_index - window_size + 1): # for each window in the batch
        x = data_set_X.iloc[index:(index + window_size + 1), :]
        x = x.values.reshape(-1)
        y = data_set_Y.iloc[index + window_size + 1, 0]
        X.append(x)
        Y.append(y)
    progression = batch_index / (len(data_set) - batch_size) * 100
    if progression - last_progression > 1:
        print(int(progression), '%')
        last_progression = progression

1 %
2 %
3 %
4 %
5 %
6 %
7 %
8 %
9 %
10 %
11 %
12 %
13 %
14 %
15 %
16 %
17 %
18 %
19 %
20 %
21 %
22 %
23 %
24 %
25 %
26 %
27 %
28 %
29 %
30 %
31 %
32 %
33 %
34 %
35 %
36 %
37 %
38 %
39 %
40 %
41 %
42 %
43 %
44 %
45 %
46 %
47 %
48 %
49 %
50 %
51 %
52 %
53 %
54 %
55 %
56 %
57 %
58 %
59 %
60 %
61 %
62 %
63 %
64 %
65 %
66 %
67 %
68 %
69 %
70 %
71 %
72 %
73 %
74 %
75 %
76 %
77 %
78 %
79 %
80 %
81 %
82 %
83 %
84 %
85 %
86 %
87 %
88 %
89 %
90 %
91 %
92 %
93 %
94 %
95 %
96 %
97 %
98 %
99 %


In [37]:
import pickle

#with open("x.pickle", "wb") as outfile:
#    pickle.dump(X, outfile)
    
#with open("y.pickle", "wb") as outfile:
#    pickle.dump(Y, outfile)